# American Express Loan Default Prediction Project

### Intro to Data Analytics (MS5600)  
Group Members:  
* Dhananjay Balakrishnan
* Siddharth Betala
* Aryamaan Saha
* Devansh Sanghvi

### Libraries and data

In [1]:
import pandas as pd
import numpy as np

In [2]:
train_data = pd.read_csv('TrainingData.csv')
test_data = pd.read_csv('testX.csv')

/tmp/ipykernel_5053/1681633734.py:1: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv('TrainingData.csv')
/tmp/ipykernel_5053/1681633734.py:2: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  test_data = pd.read_csv('testX.csv')


In [3]:
train_data.head(5)

,application_key,mvar1,mvar2,mvar3,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,...,mvar43,mvar44,mvar45,mvar46,mvar47,mvar48,mvar49,mvar50,mvar51,default_ind
0,230032,1696,1.6541,0.000,0.0,0.0,0,6015,322,40369,...,10,0.63899,na,0,C,10,770,4,3080,0
1,230033,1846,0.8095,0.000,0.0,0.0,102,7532,3171,18234,...,13,0.63836,na,na,L,732,437,5,2185,1
2,230034,1745,0.4001,0.000,0.0,0.0,missing,2536,missing,missing,...,1,1.00000,na,0,C,89,795,4,3180,1
3,230035,1739,0.2193,0.000,0.0,0.0,1982,26440,4955,20316,...,3,0.53241,0,0,L,3,755,4,3020,0
4,230036,1787,0.0118,0.225,0.0,0.0,5451,5494,5494,7987,...,1,0.92665,na,na,L,5,425,4,1700,0


In [4]:
test_data.head(5)

,application_key,mvar1,mvar2,mvar3,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,...,mvar42,mvar43,mvar44,mvar45,mvar46,mvar47,mvar48,mvar49,mvar50,mvar51
0,578069,1719,0.6174,8.623,0.0,0.0,258,258,258,10729,...,0.54545,2,0.91837,0,0,C,3247,554,4,2216
1,578070,1795,0.2051,0.000,0.0,0.0,1685,12711,8913,80519,...,0.17241,4,0.94563,0,0,C,18,554,4,2216
2,578071,1742,0.5082,0.000,0.0,0.0,1185,8954,8954,1189,...,0.64706,1,0.97054,0,0,C,11,404,4,1616
3,578072,1685,0.2595,25.409,0.0,0.0,missing,3354,missing,missing,...,0.85714,1,1.00000,na,0,C,32,528,4,2112
4,578073,1666,1.2678,0.000,0.0,0.0,570,570,570,missing,...,missing,0,0.99617,na,0,L,89,419,4,1676


# CatBoost Model

## Data preprocessing

In [5]:
train_data_new = train_data.copy()
for column in train_data:
    if column!='mvar47':
        train_data_new[column] = pd.to_numeric(train_data[column], errors ='coerce').fillna(-1).astype('float')
train_data_new.replace(-1, np.nan, inplace = True)

In [6]:
train_data_new['mvar47'] = train_data_new['mvar47'].map({'L':0, 'C':1})
train_data_new['mvar50'] = train_data_new['mvar50'].map({4:0, 5:1})
train_data_new['mvar47'] = train_data_new['mvar47'].astype(str)
train_data_new['mvar50'] = train_data_new['mvar50'].astype(str)

In [7]:
drop_cols = []
for col in train_data_new:
    if train_data_new[col].isna().sum()/len(train_data_new) > 0.5:
        print(col, 'dropped')
        train_data_new.drop(col, axis=1, inplace = True)
        drop_cols.append(col)

mvar11 dropped
mvar23 dropped
mvar31 dropped
mvar40 dropped
mvar41 dropped
mvar45 dropped


In [ ]:
#!pip install catboost

In [8]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

In [9]:
y = train_data_new['default_ind']
X = train_data_new.copy()
X.drop(['application_key','default_ind'], axis = 1, inplace = True)

cat_cols = ['mvar47','mvar50']
num_cols = []
for element in X.columns:
    if element not in cat_cols:
        num_cols.append(element)

In [10]:
count0 = (y == 0).sum()
count1 = (y==1).sum()
weights = [count1/(count0 + count1), count0/(count0 + count1)]
weights

[0.28740963855421686, 0.7125903614457831]

In [14]:
X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, random_state=1121218)
clf = CatBoostClassifier(class_weights = weights, eval_metric = 'TotalF1')
clf.fit(
    X_train, y_train,
    cat_features=cat_cols,
    eval_set=(X_val, y_val))

Learning rate set to 0.087915
0:	learn: 0.6955361	test: 0.6957900	best: 0.6957900 (0)	total: 28.2ms	remaining: 28.2s
1:	learn: 0.6967094	test: 0.7003656	best: 0.7003656 (1)	total: 50.6ms	remaining: 25.3s
2:	learn: 0.7006092	test: 0.7019034	best: 0.7019034 (2)	total: 79ms	remaining: 26.3s
3:	learn: 0.7016666	test: 0.7043273	best: 0.7043273 (3)	total: 102ms	remaining: 25.3s
4:	learn: 0.7028445	test: 0.7055447	best: 0.7055447 (4)	total: 130ms	remaining: 25.8s
5:	learn: 0.7032477	test: 0.7042911	best: 0.7055447 (4)	total: 153ms	remaining: 25.4s
6:	learn: 0.7045529	test: 0.7049009	best: 0.7055447 (4)	total: 176ms	remaining: 25s
7:	learn: 0.7050377	test: 0.7059708	best: 0.7059708 (7)	total: 208ms	remaining: 25.8s
8:	learn: 0.7054833	test: 0.7075807	best: 0.7075807 (8)	total: 242ms	remaining: 26.6s
9:	learn: 0.7061607	test: 0.7074123	best: 0.7075807 (8)	total: 272ms	remaining: 27s
10:	learn: 0.7078472	test: 0.7101219	best: 0.7101219 (10)	total: 302ms	remaining: 27.2s
11:	learn: 0.7080117	test

95:	learn: 0.7243427	test: 0.7192477	best: 0.7197350 (92)	total: 3.29s	remaining: 30.9s
96:	learn: 0.7241467	test: 0.7199558	best: 0.7199558 (96)	total: 3.32s	remaining: 30.9s
97:	learn: 0.7244723	test: 0.7194675	best: 0.7199558 (96)	total: 3.36s	remaining: 30.9s
98:	learn: 0.7246577	test: 0.7195688	best: 0.7199558 (96)	total: 3.39s	remaining: 30.9s
99:	learn: 0.7250225	test: 0.7188795	best: 0.7199558 (96)	total: 3.43s	remaining: 30.8s
100:	learn: 0.7251347	test: 0.7193670	best: 0.7199558 (96)	total: 3.46s	remaining: 30.8s
101:	learn: 0.7251684	test: 0.7184925	best: 0.7199558 (96)	total: 3.5s	remaining: 30.8s
102:	learn: 0.7253371	test: 0.7188138	best: 0.7199558 (96)	total: 3.53s	remaining: 30.8s
103:	learn: 0.7255734	test: 0.7185287	best: 0.7199558 (96)	total: 3.56s	remaining: 30.7s
104:	learn: 0.7258258	test: 0.7186632	best: 0.7199558 (96)	total: 3.6s	remaining: 30.7s
105:	learn: 0.7260107	test: 0.7187806	best: 0.7199558 (96)	total: 3.63s	remaining: 30.7s
106:	learn: 0.7260946	test: 

191:	learn: 0.7373889	test: 0.7195856	best: 0.7208026 (159)	total: 6.54s	remaining: 27.5s
192:	learn: 0.7376469	test: 0.7196028	best: 0.7208026 (159)	total: 6.58s	remaining: 27.5s
193:	learn: 0.7378266	test: 0.7198559	best: 0.7208026 (159)	total: 6.62s	remaining: 27.5s
194:	learn: 0.7379727	test: 0.7201591	best: 0.7208026 (159)	total: 6.65s	remaining: 27.5s
195:	learn: 0.7383038	test: 0.7204451	best: 0.7208026 (159)	total: 6.69s	remaining: 27.4s
196:	learn: 0.7384499	test: 0.7201755	best: 0.7208026 (159)	total: 6.73s	remaining: 27.4s
197:	learn: 0.7385170	test: 0.7200403	best: 0.7208026 (159)	total: 6.77s	remaining: 27.4s
198:	learn: 0.7386910	test: 0.7202763	best: 0.7208026 (159)	total: 6.81s	remaining: 27.4s
199:	learn: 0.7386238	test: 0.7199560	best: 0.7208026 (159)	total: 6.85s	remaining: 27.4s
200:	learn: 0.7385678	test: 0.7202763	best: 0.7208026 (159)	total: 6.88s	remaining: 27.4s
201:	learn: 0.7389271	test: 0.7200559	best: 0.7208026 (159)	total: 6.91s	remaining: 27.3s
202:	learn

289:	learn: 0.7498665	test: 0.7199898	best: 0.7220056 (266)	total: 9.82s	remaining: 24.1s
290:	learn: 0.7500067	test: 0.7198206	best: 0.7220056 (266)	total: 9.86s	remaining: 24s
291:	learn: 0.7502480	test: 0.7201415	best: 0.7220056 (266)	total: 9.88s	remaining: 24s
292:	learn: 0.7503377	test: 0.7199211	best: 0.7220056 (266)	total: 9.91s	remaining: 23.9s
293:	learn: 0.7503264	test: 0.7199211	best: 0.7220056 (266)	total: 9.94s	remaining: 23.9s
294:	learn: 0.7505285	test: 0.7200393	best: 0.7220056 (266)	total: 9.97s	remaining: 23.8s
295:	learn: 0.7506297	test: 0.7202261	best: 0.7220056 (266)	total: 10s	remaining: 23.8s
296:	learn: 0.7508709	test: 0.7205964	best: 0.7220056 (266)	total: 10s	remaining: 23.7s
297:	learn: 0.7507812	test: 0.7205118	best: 0.7220056 (266)	total: 10.1s	remaining: 23.7s
298:	learn: 0.7507419	test: 0.7205294	best: 0.7220056 (266)	total: 10.1s	remaining: 23.7s
299:	learn: 0.7510226	test: 0.7198365	best: 0.7220056 (266)	total: 10.1s	remaining: 23.7s
300:	learn: 0.7510

384:	learn: 0.7591333	test: 0.7193143	best: 0.7220056 (266)	total: 12.9s	remaining: 20.6s
385:	learn: 0.7593353	test: 0.7195819	best: 0.7220056 (266)	total: 12.9s	remaining: 20.5s
386:	learn: 0.7593353	test: 0.7195663	best: 0.7220056 (266)	total: 12.9s	remaining: 20.5s
387:	learn: 0.7596664	test: 0.7195306	best: 0.7220056 (266)	total: 13s	remaining: 20.5s
388:	learn: 0.7596495	test: 0.7195640	best: 0.7220056 (266)	total: 13s	remaining: 20.4s
389:	learn: 0.7600872	test: 0.7196154	best: 0.7220056 (266)	total: 13s	remaining: 20.4s
390:	learn: 0.7603227	test: 0.7196154	best: 0.7220056 (266)	total: 13.1s	remaining: 20.4s
391:	learn: 0.7603621	test: 0.7196309	best: 0.7220056 (266)	total: 13.1s	remaining: 20.3s
392:	learn: 0.7604410	test: 0.7195951	best: 0.7220056 (266)	total: 13.1s	remaining: 20.3s
393:	learn: 0.7606766	test: 0.7198137	best: 0.7220056 (266)	total: 13.2s	remaining: 20.2s
394:	learn: 0.7608337	test: 0.7197958	best: 0.7220056 (266)	total: 13.2s	remaining: 20.2s
395:	learn: 0.76

...

574:	learn: 0.7789170	test: 0.7209856	best: 0.7220056 (266)	total: 19.4s	remaining: 14.3s
575:	learn: 0.7790627	test: 0.7209856	best: 0.7220056 (266)	total: 19.4s	remaining: 14.3s
576:	learn: 0.7790523	test: 0.7210153	best: 0.7220056 (266)	total: 19.4s	remaining: 14.3s
577:	learn: 0.7789738	test: 0.7209301	best: 0.7220056 (266)	total: 19.5s	remaining: 14.2s
578:	learn: 0.7793435	test: 0.7208783	best: 0.7220056 (266)	total: 19.5s	remaining: 14.2s
579:	learn: 0.7794615	test: 0.7209968	best: 0.7220056 (266)	total: 19.5s	remaining: 14.1s
580:	learn: 0.7795007	test: 0.7209968	best: 0.7220056 (266)	total: 19.6s	remaining: 14.1s
581:	learn: 0.7797754	test: 0.7208597	best: 0.7220056 (266)	total: 19.6s	remaining: 14.1s
582:	learn: 0.7798208	test: 0.7208893	best: 0.7220056 (266)	total: 19.6s	remaining: 14s
583:	learn: 0.7798551	test: 0.7210967	best: 0.7220056 (266)	total: 19.7s	remaining: 14s
584:	learn: 0.7798259	test: 0.7210301	best: 0.7220056 (266)	total: 19.7s	remaining: 14s
585:	learn: 0.77

666:	learn: 0.7877906	test: 0.7198223	best: 0.7220056 (266)	total: 22.2s	remaining: 11.1s
667:	learn: 0.7879028	test: 0.7198223	best: 0.7220056 (266)	total: 22.2s	remaining: 11s
668:	learn: 0.7878857	test: 0.7199741	best: 0.7220056 (266)	total: 22.3s	remaining: 11s
669:	learn: 0.7880267	test: 0.7200407	best: 0.7220056 (266)	total: 22.3s	remaining: 11s
670:	learn: 0.7880380	test: 0.7196659	best: 0.7220056 (266)	total: 22.3s	remaining: 10.9s
671:	learn: 0.7880663	test: 0.7195850	best: 0.7220056 (266)	total: 22.3s	remaining: 10.9s
672:	learn: 0.7880776	test: 0.7194663	best: 0.7220056 (266)	total: 22.4s	remaining: 10.9s
673:	learn: 0.7879486	test: 0.7195706	best: 0.7220056 (266)	total: 22.4s	remaining: 10.8s
674:	learn: 0.7883743	test: 0.7198888	best: 0.7220056 (266)	total: 22.4s	remaining: 10.8s
675:	learn: 0.7885425	test: 0.7198888	best: 0.7220056 (266)	total: 22.5s	remaining: 10.8s
676:	learn: 0.7886831	test: 0.7198744	best: 0.7220056 (266)	total: 22.5s	remaining: 10.7s
677:	learn: 0.78

759:	learn: 0.7966466	test: 0.7182306	best: 0.7220056 (266)	total: 25s	remaining: 7.89s
760:	learn: 0.7966975	test: 0.7185153	best: 0.7220056 (266)	total: 25s	remaining: 7.86s
761:	learn: 0.7968711	test: 0.7183109	best: 0.7220056 (266)	total: 25.1s	remaining: 7.83s
762:	learn: 0.7967539	test: 0.7182863	best: 0.7220056 (266)	total: 25.1s	remaining: 7.79s
763:	learn: 0.7967760	test: 0.7183195	best: 0.7220056 (266)	total: 25.1s	remaining: 7.76s
764:	learn: 0.7967868	test: 0.7184051	best: 0.7220056 (266)	total: 25.1s	remaining: 7.72s
765:	learn: 0.7967088	test: 0.7184244	best: 0.7220056 (266)	total: 25.2s	remaining: 7.69s
766:	learn: 0.7970456	test: 0.7181536	best: 0.7220056 (266)	total: 25.2s	remaining: 7.66s
767:	learn: 0.7972863	test: 0.7184576	best: 0.7220056 (266)	total: 25.2s	remaining: 7.62s
768:	learn: 0.7973314	test: 0.7182917	best: 0.7220056 (266)	total: 25.3s	remaining: 7.59s
769:	learn: 0.7975951	test: 0.7181204	best: 0.7220056 (266)	total: 25.3s	remaining: 7.55s
770:	learn: 0.

851:	learn: 0.8052260	test: 0.7172175	best: 0.7220056 (266)	total: 27.7s	remaining: 4.81s
852:	learn: 0.8053329	test: 0.7175276	best: 0.7220056 (266)	total: 27.8s	remaining: 4.78s
853:	learn: 0.8052991	test: 0.7175472	best: 0.7220056 (266)	total: 27.8s	remaining: 4.75s
854:	learn: 0.8055225	test: 0.7173153	best: 0.7220056 (266)	total: 27.8s	remaining: 4.72s
855:	learn: 0.8055902	test: 0.7173484	best: 0.7220056 (266)	total: 27.8s	remaining: 4.68s
856:	learn: 0.8054500	test: 0.7175608	best: 0.7220056 (266)	total: 27.9s	remaining: 4.65s
857:	learn: 0.8055783	test: 0.7175472	best: 0.7220056 (266)	total: 27.9s	remaining: 4.62s
858:	learn: 0.8056739	test: 0.7179371	best: 0.7220056 (266)	total: 27.9s	remaining: 4.58s
859:	learn: 0.8055160	test: 0.7174087	best: 0.7220056 (266)	total: 28s	remaining: 4.55s
860:	learn: 0.8053431	test: 0.7174223	best: 0.7220056 (266)	total: 28s	remaining: 4.52s
861:	learn: 0.8054269	test: 0.7173696	best: 0.7220056 (266)	total: 28s	remaining: 4.49s
862:	learn: 0.80

943:	learn: 0.8117893	test: 0.7165784	best: 0.7220056 (266)	total: 30.5s	remaining: 1.81s
944:	learn: 0.8118339	test: 0.7163204	best: 0.7220056 (266)	total: 30.5s	remaining: 1.77s
945:	learn: 0.8118006	test: 0.7167437	best: 0.7220056 (266)	total: 30.5s	remaining: 1.74s
946:	learn: 0.8118285	test: 0.7169289	best: 0.7220056 (266)	total: 30.6s	remaining: 1.71s
947:	learn: 0.8117609	test: 0.7170480	best: 0.7220056 (266)	total: 30.6s	remaining: 1.68s
948:	learn: 0.8119515	test: 0.7166180	best: 0.7220056 (266)	total: 30.6s	remaining: 1.65s
949:	learn: 0.8119741	test: 0.7167900	best: 0.7220056 (266)	total: 30.6s	remaining: 1.61s
950:	learn: 0.8119907	test: 0.7165122	best: 0.7220056 (266)	total: 30.7s	remaining: 1.58s
951:	learn: 0.8120751	test: 0.7165914	best: 0.7220056 (266)	total: 30.7s	remaining: 1.55s
952:	learn: 0.8120981	test: 0.7164261	best: 0.7220056 (266)	total: 30.7s	remaining: 1.52s
953:	learn: 0.8121937	test: 0.7162939	best: 0.7220056 (266)	total: 30.8s	remaining: 1.48s
954:	learn

## Test set predictions using Catboost

### Preprocessing test set

In [15]:
test_data_new = test_data.copy()
for column in test_data:
    if column!='mvar47':
        test_data_new[column] = pd.to_numeric(test_data[column], errors ='coerce').fillna(-1).astype('float')
test_data_new.replace(-1, np.nan, inplace = True)

test_data_new['mvar47'] = test_data_new['mvar47'].map({'L':0, 'C':1})
test_data_new['mvar50'] = test_data_new['mvar50'].map({4:0, 5:1})
test_data_new['mvar47'] = test_data_new['mvar47'].astype(str)
test_data_new['mvar50'] = test_data_new['mvar50'].astype(str)
test_data_new.drop(drop_cols, axis=1, inplace = True)

X_test = test_data_new.copy()
X_test.drop(['application_key'], axis = 1, inplace = True)

### Predictions

In [100]:
test_preds_catboost = clf.predict(X_test)
keys = test_data_new['application_key']
df_out = pd.DataFrame(list(zip(keys, test_preds_catboost)), columns =['keys', 'predictions'], dtype = int)

In [101]:
df_out.to_csv('csvs/mostly_just_bt_catbooost.csv', header=False, index=False)

# XGBoost Model

In [20]:
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

In [27]:
train_data = pd.read_csv('TrainingData.csv')
test_data = pd.read_csv('testX.csv')

/tmp/ipykernel_5053/1681633734.py:1: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv('TrainingData.csv')
/tmp/ipykernel_5053/1681633734.py:2: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  test_data = pd.read_csv('testX.csv')


## Data preprocessing

In [28]:
train_data_new = train_data.copy()
for column in train_data:
    if column!='mvar47':
        train_data_new[column] = pd.to_numeric(train_data[column], errors ='coerce').fillna(-1).astype('float')
train_data_new.replace(-1, np.nan, inplace = True)

train_data_new['mvar47'] = train_data_new['mvar47'].map({'L':0, 'C':1})
train_data_new['mvar50'] = train_data_new['mvar50'].map({4:0, 5:1})
train_data_new['mvar47'] = train_data_new['mvar47'].astype(str)
train_data_new['mvar50'] = train_data_new['mvar50'].astype(str)

In [29]:
drop_cols = []
for col in train_data_new:
    if train_data_new[col].isna().sum()/len(train_data_new) > 0.5:
        print(col, 'dropped')
        train_data_new.drop(col, axis=1, inplace = True)
        drop_cols.append(col)

mvar11 dropped
mvar23 dropped
mvar31 dropped
mvar40 dropped
mvar41 dropped
mvar45 dropped


In [30]:
y = train_data_new['default_ind']
X = train_data_new.copy()
count0 = (y == 0).sum()
count1 = (y==1).sum()
scale = count0/count1
X.drop(['application_key','default_ind'], axis = 1, inplace = True)

In [31]:
cat_cols = ['mvar47','mvar50']
num_cols = []
for element in X.columns:
    if element not in cat_cols:
        num_cols.append(element)
        
for col in num_cols:
    X[col].fillna(-999, inplace = True)

for col in cat_cols:
    X[col].fillna(-999, inplace = True)

In [32]:
ct = ColumnTransformer([('somename', StandardScaler(), num_cols)], remainder='passthrough')
X_scaled = ct.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, stratify=y, random_state=1)

In [33]:
from sklearn.model_selection import GridSearchCV
PARAMETERS = {"subsample":[0.5, 0.75, 1], "colsample_bytree":[0.5, 0.75, 1],
              "max_depth":[2, 6, 12], "min_child_weight":[1,5,15], 
              "learning_rate":[0.3, 0.1, 0.03], "n_estimators":[100]}


model = xgb.XGBClassifier(n_estimators=100, n_jobs=-1)
model_gs = GridSearchCV(model,param_grid=PARAMETERS,cv=3,scoring="f1")

# model_gs.fit(X_train,y_train,early_stopping_rounds=10, eval_metric="error",eval_set=[(X_test, y_test)],verbose=1)
# print(model_gs.best_params_)
#best_params = model_gs.best_params_

best_params = {'colsample_bytree': 1, 'learning_rate': 0.1, 
               'max_depth': 6, 'min_child_weight': 15, 'n_estimators': 100, 'subsample': 0.5}

In [34]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
xgb_cl = xgb.XGBClassifier(**best_params, scale_pos_weight = scale)
xgb_cl.fit(X_train, y_train)
preds = xgb_cl.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, preds)}, F1: {f1_score(y_test, preds)}')
confusion_matrix(y_test, preds)

Accuracy: 0.7267951807228916, F1: 0.5945791318029036


array([[10924,  3862],
       [ 1807,  4157]])

## Test set predictions using XGBoost

### Test set preprocessing

In [36]:
test_data_new = test_data.copy()
for column in test_data:
    if column!='mvar47':
        test_data_new[column] = pd.to_numeric(test_data[column], errors ='coerce').fillna(-1).astype('float')
test_data_new.replace(-1, np.nan, inplace = True)

test_data_new['mvar47'] = test_data_new['mvar47'].map({'L':0, 'C':1})
test_data_new['mvar50'] = test_data_new['mvar50'].map({4:0, 5:1})
test_data_new['mvar47'] = test_data_new['mvar47'].astype(str)
test_data_new['mvar50'] = test_data_new['mvar50'].astype(str)
test_data_new.drop(drop_cols, axis=1, inplace = True)

X_test = test_data_new.copy()
X_test.drop(['application_key'], axis = 1, inplace = True)

for col in num_cols:
    X[col].fillna(-999, inplace = True)

for col in cat_cols:
    X[col].fillna(-999, inplace = True)

In [37]:
ct = ColumnTransformer([('somename', StandardScaler(), num_cols)], remainder='passthrough')
X_test_scaled = ct.fit_transform(X_test)

### Predictions

In [102]:
test_preds_xgb = xgb_cl.predict(X_test_scaled)
keys = test_data_new['application_key']
df_out = pd.DataFrame(list(zip(keys, test_preds_xgb)), columns =['keys', 'predictions'], dtype = int)

In [103]:
df_out.to_csv('csvs/mostly_just_bt_xgb.csv', header=False, index=False)

# Artificial Neural Network Model

In [40]:
train_data = pd.read_csv('TrainingData.csv')
test_data = pd.read_csv('testX.csv')

/tmp/ipykernel_5053/1681633734.py:1: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv('TrainingData.csv')
/tmp/ipykernel_5053/1681633734.py:2: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  test_data = pd.read_csv('testX.csv')


## Data preprocessing

In [41]:
train_data = train_data.replace(['na', 'missing', '#VALUE!', np.inf, -np.inf], np.NaN)
train_data['mvar47'] = train_data['mvar47'].map({'L':0, 'C':1})
train_data['mvar50'] = train_data['mvar50'].map({4:0, 5:1})

drop_cols = []
for col in train_data:
    if train_data[col].isna().sum()/len(train_data) > 0.3:
        print(col, 'dropped')
        train_data.drop(col, axis=1, inplace = True)
        drop_cols.append(col)

mvar11 dropped
mvar15 dropped
mvar22 dropped
mvar23 dropped
mvar30 dropped
mvar31 dropped
mvar35 dropped
mvar40 dropped
mvar41 dropped
mvar45 dropped
mvar50 dropped


In [42]:
X_train = train_data.drop(['application_key', 'default_ind'], axis=1)
y_train = train_data['default_ind'].copy()

In [43]:
X_test = test_data.copy()
X_test = X_test.replace(['na', 'missing', '#VALUE!', np.inf, -np.inf], np.NaN)
X_test['mvar47'] = X_test['mvar47'].map({'L':0, 'C':1})
X_test['mvar50'] = X_test['mvar50'].map({4:0, 5:1})
X_test = X_test.drop(drop_cols, axis=1)
X_test = X_test.drop(['application_key'], axis=1)
X_test

,mvar1,mvar2,mvar3,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,mvar10,...,mvar38,mvar39,mvar42,mvar43,mvar44,mvar46,mvar47,mvar48,mvar49,mvar51
0,1719,0.6174,8.623,0.000,0.000,258,258,258,10729,307,...,2,0,0.54545,2,0.91837,0,1,3247,554,2216
1,1795,0.2051,0.000,0.000,0.000,1685,12711,8913,80519,18099,...,4,0,0.17241,4,0.94563,0,1,18,554,2216
2,1742,0.5082,0.000,0.000,0.000,1185,8954,8954,1189,1185,...,1,0,0.64706,1,0.97054,0,1,11,404,1616
3,1685,0.2595,25.409,0.000,0.000,NaN,3354,NaN,NaN,3354,...,2,0,0.85714,1,1.00000,0,1,32,528,2112
4,1666,1.2678,0.000,0.000,0.000,570,570,570,NaN,570,...,2,0,NaN,0,0.99617,0,0,89,419,1676
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46995,1736,2.1740,0.000,0.000,0.000,11,4248,1577,13379,6671,...,4,0,0,4,0.43829,0,1,19,383,1532
46996,1724,0.0000,1.108,0.768,0.000,NaN,64041,NaN,10926,84839,...,5,0,0.16667,14,0.57931,0,1,5,595,2380
46997,1605,0.2901,11.561,0.937,2.976,NaN,2277,NaN,3964,5709,...,5,2,0.5,4,0.42069,0,1,22,835,3340
46998,1780,1.1874,0.000,0.000,0.000,0,6356,4802,3206,18180,...,8,0,0.06061,9,0.53251,0,0,9,449,1796


### Missing value imputation with median and mode

In [44]:
for col in X_train.columns:
    if col != 'mvar47':
        colmed = X_train[col].median()
        X_train[col] = X_train[col].fillna(colmed)
        X_test[col] = X_test[col].fillna(colmed)
    else:
        colmode = X_train[col].mode()[0]
        X_train[col] = X_train[col].fillna(colmode)
        X_test[col] = X_test[col].fillna(colmode)

In [45]:
for col in X_train.columns:
    X_train[col] = pd.to_numeric(X_train[col])
    X_test[col] = pd.to_numeric(X_test[col])    

### Feature removal using VIF

In [46]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [47]:
temp = X_train.drop(['mvar47'], axis=1)
vif_data = pd.DataFrame()
vif_data["feature"] = temp.columns
vif_data["VIF"] = [variance_inflation_factor(temp.values, i)
                          for i in range(len(temp.columns))]
print(vif_data)

   feature         VIF
0    mvar1   55.433365
1    mvar2    1.705317
2    mvar3    3.449780
3    mvar4    5.192754
4    mvar5    1.835700
5    mvar6    1.819580
6    mvar7    7.253896
7    mvar8    3.464413
8    mvar9    5.517938
9   mvar10   10.118730
10  mvar12    2.239803
11  mvar13    1.405205
12  mvar14    1.001018
13  mvar16    9.115965
14  mvar17   66.074893
15  mvar18   57.981871
16  mvar19   52.507427
17  mvar20   53.741230
18  mvar21    9.119847
19  mvar24    1.094294
20  mvar25    9.643429
21  mvar26   15.286277
22  mvar27   16.420945
23  mvar28    2.787053
24  mvar29    9.511366
25  mvar32   16.001793
26  mvar33    2.320239
27  mvar34    2.119680
28  mvar36    6.859268
29  mvar37    9.903616
30  mvar38    4.335002
31  mvar39    1.778563
32  mvar42    4.020594
33  mvar43    7.769519
34  mvar44   13.941430
35  mvar46    4.927243
36  mvar48    1.252699
37  mvar49  154.788319
38  mvar51  153.661364


In [48]:
temp = X_train.drop(['mvar47', 'mvar49'], axis=1)
vif_data = pd.DataFrame()
vif_data["feature"] = temp.columns
vif_data["VIF"] = [variance_inflation_factor(temp.values, i)
                          for i in range(len(temp.columns))]
print(vif_data)

   feature        VIF
0    mvar1  54.878629
1    mvar2   1.699425
2    mvar3   3.448699
3    mvar4   5.192522
4    mvar5   1.835320
5    mvar6   1.817069
6    mvar7   7.239201
7    mvar8   3.440276
8    mvar9   5.504767
9   mvar10  10.086244
10  mvar12   2.239183
11  mvar13   1.405197
12  mvar14   1.000917
13  mvar16   9.115932
14  mvar17  66.061159
15  mvar18  57.978800
16  mvar19  52.484788
17  mvar20  53.738595
18  mvar21   8.995140
19  mvar24   1.094064
20  mvar25   9.638735
21  mvar26  15.279846
22  mvar27  16.420935
23  mvar28   2.501901
24  mvar29   9.511359
25  mvar32  16.001742
26  mvar33   2.319601
27  mvar34   2.118999
28  mvar36   6.858685
29  mvar37   9.902215
30  mvar38   4.325141
31  mvar39   1.778538
32  mvar42   3.963052
33  mvar43   7.749399
34  mvar44  13.939862
35  mvar46   4.927090
36  mvar48   1.252653
37  mvar51  17.992589


In [49]:
temp = X_train.drop(['mvar47', 'mvar49', 'mvar17'], axis=1)
vif_data = pd.DataFrame()
vif_data["feature"] = temp.columns
vif_data["VIF"] = [variance_inflation_factor(temp.values, i)
                          for i in range(len(temp.columns))]
print(vif_data)

   feature        VIF
0    mvar1  54.875388
1    mvar2   1.699366
2    mvar3   3.448687
3    mvar4   5.192167
4    mvar5   1.835301
5    mvar6   1.817053
6    mvar7   7.238906
7    mvar8   3.440220
8    mvar9   5.504693
9   mvar10  10.085246
10  mvar12   2.237694
11  mvar13   1.405192
12  mvar14   1.000828
13  mvar16   8.010596
14  mvar18   8.855943
15  mvar19  46.955874
16  mvar20  48.742018
17  mvar21   8.987925
18  mvar24   1.094001
19  mvar25   9.638559
20  mvar26  15.279005
21  mvar27  16.419422
22  mvar28   2.501829
23  mvar29   9.511093
24  mvar32  16.001501
25  mvar33   2.319350
26  mvar34   2.117304
27  mvar36   6.858390
28  mvar37   9.902179
29  mvar38   4.324830
30  mvar39   1.778520
31  mvar42   3.962463
32  mvar43   7.749392
33  mvar44  13.939150
34  mvar46   4.926836
35  mvar48   1.252652
36  mvar51  17.991326


In [50]:
temp = X_train.drop(['mvar47', 'mvar49', 'mvar17', 'mvar1'], axis=1)
vif_data = pd.DataFrame()
vif_data["feature"] = temp.columns
vif_data["VIF"] = [variance_inflation_factor(temp.values, i)
                          for i in range(len(temp.columns))]
print(vif_data)

   feature        VIF
0    mvar2   1.689069
1    mvar3   3.413486
2    mvar4   5.191709
3    mvar5   1.829784
4    mvar6   1.814093
5    mvar7   7.236856
6    mvar8   3.369797
7    mvar9   5.464207
8   mvar10  10.079771
9   mvar12   2.235604
10  mvar13   1.404967
11  mvar14   1.000804
12  mvar16   8.007606
13  mvar18   8.855092
14  mvar19  46.950332
15  mvar20  48.742004
16  mvar21   7.737317
17  mvar24   1.093996
18  mvar25   9.512284
19  mvar26  15.265338
20  mvar27  16.383281
21  mvar28   2.357192
22  mvar29   9.206958
23  mvar32  15.703660
24  mvar33   2.315631
25  mvar34   2.107952
26  mvar36   6.777286
27  mvar37   9.698382
28  mvar38   4.308802
29  mvar39   1.776096
30  mvar42   3.772616
31  mvar43   7.172008
32  mvar44   9.981438
33  mvar46   4.926489
34  mvar48   1.247169
35  mvar51  12.730746


In [51]:
temp = X_train.drop(['mvar47', 'mvar49', 'mvar17', 'mvar1', 'mvar20'], axis=1)
vif_data = pd.DataFrame()
vif_data["feature"] = temp.columns
vif_data["VIF"] = [variance_inflation_factor(temp.values, i)
                          for i in range(len(temp.columns))]
print(vif_data)

   feature        VIF
0    mvar2   1.688463
1    mvar3   3.413215
2    mvar4   5.191674
3    mvar5   1.829714
4    mvar6   1.813908
5    mvar7   7.231226
6    mvar8   3.366470
7    mvar9   5.464194
8   mvar10  10.063263
9   mvar12   2.230321
10  mvar13   1.400028
11  mvar14   1.000803
12  mvar16   7.897554
13  mvar18   8.709214
14  mvar19   3.138205
15  mvar21   7.715495
16  mvar24   1.093969
17  mvar25   9.505879
18  mvar26  15.265277
19  mvar27  16.382326
20  mvar28   2.357006
21  mvar29   9.205725
22  mvar32  15.702708
23  mvar33   2.310591
24  mvar34   2.107017
25  mvar36   6.768750
26  mvar37   9.697207
27  mvar38   4.305608
28  mvar39   1.775883
29  mvar42   3.768358
30  mvar43   7.169887
31  mvar44   9.974321
32  mvar46   4.926456
33  mvar48   1.247112
34  mvar51  12.730684


In [52]:
temp = X_train.drop(['mvar47', 'mvar49', 'mvar17', 'mvar1', 'mvar20', 'mvar27'], axis=1)
vif_data = pd.DataFrame()
vif_data["feature"] = temp.columns
vif_data["VIF"] = [variance_inflation_factor(temp.values, i)
                          for i in range(len(temp.columns))]
print(vif_data)

   feature        VIF
0    mvar2   1.688425
1    mvar3   3.411943
2    mvar4   5.191587
3    mvar5   1.829646
4    mvar6   1.813693
5    mvar7   7.217228
6    mvar8   3.364214
7    mvar9   5.454208
8   mvar10  10.037661
9   mvar12   2.230286
10  mvar13   1.399999
11  mvar14   1.000792
12  mvar16   7.897518
13  mvar18   8.708930
14  mvar19   3.138193
15  mvar21   7.714965
16  mvar24   1.093833
17  mvar25   8.809589
18  mvar26   6.503154
19  mvar28   2.356964
20  mvar29   8.915536
21  mvar32  15.452201
22  mvar33   2.310210
23  mvar34   2.107011
24  mvar36   6.762966
25  mvar37   9.674288
26  mvar38   4.304769
27  mvar39   1.775858
28  mvar42   3.768340
29  mvar43   7.168712
30  mvar44   9.956707
31  mvar46   4.926418
32  mvar48   1.247103
33  mvar51  12.716791


In [53]:
temp = X_train.drop(['mvar47', 'mvar49', 'mvar17', 'mvar1', 'mvar20', 'mvar27', 'mvar32'], axis=1)
vif_data = pd.DataFrame()
vif_data["feature"] = temp.columns
vif_data["VIF"] = [variance_inflation_factor(temp.values, i)
                          for i in range(len(temp.columns))]
print(vif_data)

   feature        VIF
0    mvar2   1.687730
1    mvar3   3.411660
2    mvar4   5.190107
3    mvar5   1.829456
4    mvar6   1.755855
5    mvar7   7.196293
6    mvar8   3.349790
7    mvar9   5.262475
8   mvar10  10.001278
9   mvar12   2.215918
10  mvar13   1.399424
11  mvar14   1.000781
12  mvar16   7.880905
13  mvar18   8.700023
14  mvar19   3.110905
15  mvar21   7.563636
16  mvar24   1.092861
17  mvar25   7.376086
18  mvar26   6.333168
19  mvar28   2.354118
20  mvar29   8.848352
21  mvar33   2.310146
22  mvar34   2.106336
23  mvar36   6.746969
24  mvar37   4.579740
25  mvar38   4.303567
26  mvar39   1.775722
27  mvar42   3.759126
28  mvar43   7.135286
29  mvar44   9.863834
30  mvar46   4.925965
31  mvar48   1.247038
32  mvar51  12.624345


In [54]:
vifcols = ['mvar49', 'mvar17', 'mvar1', 'mvar20', 'mvar27', 'mvar32']
X_train = X_train.drop(vifcols, axis=1)
X_test = X_test.drop(vifcols, axis=1)

### Neural Net architecture and training

In [58]:
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint

In [59]:
count0 = (y_train == 0).sum()
count1 = (y_train==1).sum()
weights = [count1/(count0 + count1), count0/(count0 + count1)]
weights

[0.28740963855421686, 0.7125903614457831]

In [60]:
def f1score(y_true, y_pred): 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [62]:
model = Sequential()
model.add(Dense(64, input_shape=(len(X_train.columns),), activation='relu'))
model.add(BatchNormalization())
model.add(Dense(16, input_shape=(64,), activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', f1score])
callbacks = [ModelCheckpoint('./ckpt/weights.{epoch:02d}-{val_f1score:.4f}.hdf5', monitor='val_f1score', 
                             save_weights_only=True, save_best_only=True, mode='max', verbose=1)]

In [63]:
model.fit(x=X_train, y=pd.DataFrame(y_train), epochs=100, batch_size=32, verbose=0,
          validation_split=0.15, class_weight = {0:weights[0], 1:weights[1]}, callbacks=callbacks)

In [64]:
model.load_weights('./ckpt/weights.71-0.54.hdf5')
y_pred = (model.predict(X_test) > 0.5).astype("int32")

## Test set predictions

In [65]:
output = pd.DataFrame(y_pred, index=test_data['application_key'])
output.to_csv('csvs/mostly_just_bt_nn.csv', header=False, index=True)

# Histogram Gradient Boosting Model

In [81]:
train_data = pd.read_csv('TrainingData.csv')
test_data = pd.read_csv('testX.csv')

/tmp/ipykernel_5053/1681633734.py:1: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv('TrainingData.csv')
/tmp/ipykernel_5053/1681633734.py:2: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  test_data = pd.read_csv('testX.csv')


## Data preprocessing

In [82]:
train_data.replace(['na', 'missing', '#VALUE!', np.inf, -np.inf], np.NaN, inplace = True)
train_data_new = train_data.copy()
for column in train_data:
    if column!='mvar47':
        train_data_new[column] = pd.to_numeric(train_data[column], errors ='coerce').fillna(-1).astype('float')
train_data_new.replace(-1, np.nan, inplace = True)

train_data_new['mvar47'] = train_data_new['mvar47'].map({'L':0, 'C':1})
train_data_new['mvar50'] = train_data_new['mvar50'].map({4:0, 5:1})

In [83]:
seed = 2019
drop_cols = []
for col in train_data_new:
    if train_data_new[col].isna().sum()/len(train_data_new) > 0.5:
        print(col, 'dropped')
        train_data_new.drop(col, axis=1, inplace = True)
        drop_cols.append(col)

mvar11 dropped
mvar23 dropped
mvar31 dropped
mvar40 dropped
mvar41 dropped
mvar45 dropped


In [84]:
y = train_data_new['default_ind']
X = train_data_new.copy()
X.drop(['application_key','default_ind'], axis = 1, inplace = True)

In [85]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, random_state=1121218)

In [86]:
cat_cols = ['mvar47','mvar50']
num_cols = []
for element in X.columns:
    if element not in cat_cols:
        num_cols.append(element)

In [87]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
numeric_transformer = make_pipeline(StandardScaler())
categorical_transformer = make_pipeline(OneHotEncoder())
transf = [('numeric_transformer', numeric_transformer, num_cols),('categorical_transformer',categorical_transformer, cat_cols)]
Preprocessor = ColumnTransformer(transformers=transf)
X_train = pd.get_dummies(X_train, ['mvar47','mvar50'], columns = ['mvar47','mvar50'])

In [88]:
from sklearn.ensemble import HistGradientBoostingClassifier
hgb_pipe = make_pipeline(Preprocessor,HistGradientBoostingClassifier())
params = {
    'histgradientboostingclassifier__max_iter':[100,200,300,400,500],
    'histgradientboostingclassifier__learning_rate':[0.001,0.01,0.1,0.2,0.5,0.8,1],
    'histgradientboostingclassifier__l2_regularization':[0,0.5,1,1.5,2,5,10,50,100,500,1000],
    'histgradientboostingclassifier__scoring': ['f1'],
    'histgradientboostingclassifier__random_state' : [seed],
}

from sklearn.model_selection import GridSearchCV
hgb_grid = GridSearchCV(hgb_pipe, params, n_jobs=5, cv=5, scoring='f1',verbose=0, refit=True)
hgb_grid.fit(X_train, y_train)

In [94]:
# print(hgb_grid.best_params_)
# print(hgb_grid.best_score_)
histgb = HistGradientBoostingClassifier(learning_rate = 0.2, 
                                        l2_regularization=50, random_state = 2019, verbose = 4)
histgb.fit(X_train,y_train)

Binning 0.021 GB of training data: 0.180 s
Binning 0.002 GB of validation data: 0.012 s
Fitting gradient boosted rounds:
[1/100] 1 tree, 31 leaves, max depth = 7, train loss: 0.56810, val loss: 0.56889, in 0.021s
[2/100] 1 tree, 31 leaves, max depth = 8, train loss: 0.54655, val loss: 0.54785, in 0.027s
[3/100] 1 tree, 31 leaves, max depth = 7, train loss: 0.53107, val loss: 0.53330, in 0.022s
[4/100] 1 tree, 31 leaves, max depth = 6, train loss: 0.51934, val loss: 0.52237, in 0.021s
[5/100] 1 tree, 31 leaves, max depth = 7, train loss: 0.51063, val loss: 0.51452, in 0.021s
[6/100] 1 tree, 31 leaves, max depth = 9, train loss: 0.50389, val loss: 0.50838, in 0.013s
[7/100] 1 tree, 31 leaves, max depth = 9, train loss: 0.49814, val loss: 0.50348, in 0.013s
[8/100] 1 tree, 31 leaves, max depth = 8, train loss: 0.49351, val loss: 0.49917, in 0.020s
[9/100] 1 tree, 31 leaves, max depth = 11, train loss: 0.48980, val loss: 0.49627, in 0.013s
[10/100] 1 tree, 31 leaves, max depth = 8, train l

HistGradientBoostingClassifier(l2_regularization=50, learning_rate=0.2,
                               random_state=2019, verbose=4)

In [95]:
from sklearn.metrics import f1_score
X_val = pd.get_dummies(X_val, ['mvar47','mvar50'], columns = ['mvar47','mvar50'])
y_pred_hgb = histgb.predict(X_val)
print(f1_score(y_val, y_pred_hgb))

0.5258263418578794


## Test set predictions using HistgradientBoost

### Test set preprocessing

In [96]:
test_data_new = test_data.copy()
for column in test_data:
    if column!='mvar47':
        test_data_new[column] = pd.to_numeric(test_data[column], errors ='coerce').fillna(-1).astype('float')
test_data_new.replace(-1, np.nan, inplace = True)

test_data_new['mvar47'] = test_data_new['mvar47'].map({'L':0, 'C':1})
test_data_new['mvar50'] = test_data_new['mvar50'].map({4:0, 5:1})
test_data_new.drop(drop_cols, axis=1, inplace = True)

X_test = test_data_new.copy()
X_test.drop(['application_key'], axis = 1, inplace = True)

### Predictions

In [97]:
X_test = pd.get_dummies(X_test, ['mvar47','mvar50'], columns = ['mvar47','mvar50'])
test_pred_histgb = histgb.predict(X_test)

In [105]:
keys = test_data_new['application_key']
df_out = pd.DataFrame(list(zip(keys, test_pred_histgb)), columns =['keys', 'predictions'], dtype = int)

In [106]:
df_out.to_csv('csvs/mostly_just_bt_histgb.csv', header=False, index=False)

# Bagging Models

### Bagging Catboost, XGBoost, and Neural Network models

In [125]:
catb = pd.read_csv('csvs/mostly_just_bt_catboost.csv', header = None)
xgb = pd.read_csv('csvs/mostly_just_bt_xgb.csv', header = None)
nn = pd.read_csv('csvs/mostly_just_bt_nn.csv', header = None)

In [126]:
full = pd.DataFrame({
    'keys':catb[0],
    'catb':catb[1],
    'xgb': xgb[1],
    'nn':nn[1],
}).set_index('keys')
full

,catb,xgb,nn
keys,,,
578069,0,1,1
578070,0,0,0
578071,0,0,0
578072,1,1,1
578073,0,1,0
...,...,...,...
310027,1,1,1
310028,0,0,0
310029,1,1,1


In [127]:
mode = full.mode(axis=1)
full['vote'] = mode[0]
full[['vote']]

,vote
keys,
578069,1
578070,0
578071,0
578072,1
578073,0
...,...
310027,1
310028,0
310029,1


In [128]:
full[['vote']].to_csv('csvs/mostly_just_bt_bagging_3.csv', header=False, index=True)

### Bagging Catboost, XGBoost, HistGB, and Neural Network models

In [112]:
catb = pd.read_csv('csvs/mostly_just_bt_catboost.csv', header = None)
xgb = pd.read_csv('csvs/mostly_just_bt_xgb.csv', header = None)
histgb = pd.read_csv('csvs/mostly_just_bt_histgb.csv', header = None)
nn = pd.read_csv('csvs/mostly_just_bt_nn.csv', header = None)

In [113]:
full = pd.DataFrame({
    'keys':catb[0],
    'catb':catb[1],
    'xgb': xgb[1],
    'histgb':histgb[1],
    'nn':nn[1],
}).set_index('keys')
full

,catb,xgb,histgb,nn
keys,,,,
578069,0,1,0,1
578070,0,0,0,0
578071,0,0,0,0
578072,1,1,0,1
578073,0,1,0,0
...,...,...,...,...
310027,1,1,0,1
310028,0,0,0,0
310029,1,1,1,1


In [114]:
mode = full.mode(axis=1)

In [115]:
full['vote'] = np.where(mode.isna().any(1), mode[0], np.nan)
full['vote'] = full['vote'].fillna(full.pop('catb'))
full['vote'] = full['vote'].astype(np.int32)
full[['vote']]

,vote
keys,
578069,0
578070,0
578071,0
578072,1
578073,0
...,...
310027,1
310028,0
310029,1


In [116]:
full[['vote']].to_csv('csvs/mostly_just_bt_bagging_4.csv', header=False, index=True)

### Voting Classifier with HistGB and Catboost

In [129]:
train_data_new = train_data.copy()
for column in train_data:
    if column!='mvar47':
        train_data_new[column] = pd.to_numeric(train_data[column], errors ='coerce').fillna(-1).astype('float')
train_data_new.replace(-1, np.nan, inplace = True)

In [130]:
train_data_new['mvar47'] = train_data_new['mvar47'].map({'L':0, 'C':1})
train_data_new['mvar50'] = train_data_new['mvar50'].map({4:0, 5:1})
train_data_new['mvar47'] = train_data_new['mvar47'].astype(str)
train_data_new['mvar50'] = train_data_new['mvar50'].astype(str)

In [131]:
drop_cols = []
for col in train_data_new:
    if train_data_new[col].isna().sum()/len(train_data_new) > 0.5:
        print(col, 'dropped')
        train_data_new.drop(col, axis=1, inplace = True)
        drop_cols.append(col)

mvar11 dropped
mvar23 dropped
mvar31 dropped
mvar40 dropped
mvar41 dropped
mvar45 dropped


In [132]:
y = train_data_new['default_ind']
X = train_data_new.copy()
X.drop(['application_key','default_ind'], axis = 1, inplace = True)

cat_cols = ['mvar47','mvar50']
num_cols = []
for element in X.columns:
    if element not in cat_cols:
        num_cols.append(element)

In [133]:
count0 = (y == 0).sum()
count1 = (y==1).sum()
weights = [count1/(count0 + count1), count0/(count0 + count1)]
weights

[0.28740963855421686, 0.7125903614457831]

In [136]:
from catboost import CatBoostClassifier
from sklearn.ensemble import HistGradientBoostingClassifier, VotingClassifier
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, random_state=1121218)

In [137]:
eclf1 = HistGradientBoostingClassifier()
eclf2 = CatBoostClassifier(class_weights = weights, eval_metric = 'TotalF1')
vc = VotingClassifier(estimators = [('histgb', eclf1), ('catb', eclf2)], voting = 'soft')
vc.fit(X_train,y_train)

Learning rate set to 0.060122
0:	learn: 0.6914945	total: 42.7ms	remaining: 42.7s
1:	learn: 0.6957368	total: 52.1ms	remaining: 26s
2:	learn: 0.6964263	total: 61.5ms	remaining: 20.4s
3:	learn: 0.6982363	total: 71.4ms	remaining: 17.8s
4:	learn: 0.7000821	total: 81.3ms	remaining: 16.2s
5:	learn: 0.7014157	total: 91.1ms	remaining: 15.1s
6:	learn: 0.7022322	total: 100ms	remaining: 14.2s
7:	learn: 0.7021574	total: 110ms	remaining: 13.7s
8:	learn: 0.7017975	total: 120ms	remaining: 13.2s
9:	learn: 0.7036433	total: 129ms	remaining: 12.8s
10:	learn: 0.7041265	total: 139ms	remaining: 12.5s
11:	learn: 0.7050813	total: 148ms	remaining: 12.1s
12:	learn: 0.7059234	total: 157ms	remaining: 11.9s
13:	learn: 0.7067875	total: 166ms	remaining: 11.7s
14:	learn: 0.7064561	total: 176ms	remaining: 11.5s
15:	learn: 0.7074971	total: 185ms	remaining: 11.4s
16:	learn: 0.7080118	total: 197ms	remaining: 11.4s
17:	learn: 0.7078071	total: 208ms	remaining: 11.3s
18:	learn: 0.7076320	total: 217ms	remaining: 11.2s
19:	lea

178:	learn: 0.7287753	total: 1.83s	remaining: 8.41s
179:	learn: 0.7286803	total: 1.84s	remaining: 8.41s
180:	learn: 0.7287195	total: 1.85s	remaining: 8.4s
181:	learn: 0.7288765	total: 1.86s	remaining: 8.38s
182:	learn: 0.7288765	total: 1.87s	remaining: 8.36s
183:	learn: 0.7288882	total: 1.88s	remaining: 8.35s
184:	learn: 0.7290452	total: 1.89s	remaining: 8.33s
185:	learn: 0.7292027	total: 1.9s	remaining: 8.31s
186:	learn: 0.7293767	total: 1.91s	remaining: 8.3s
187:	learn: 0.7295117	total: 1.92s	remaining: 8.29s
188:	learn: 0.7296852	total: 1.93s	remaining: 8.28s
189:	learn: 0.7297580	total: 1.94s	remaining: 8.26s
190:	learn: 0.7297922	total: 1.95s	remaining: 8.25s
191:	learn: 0.7298930	total: 1.96s	remaining: 8.24s
192:	learn: 0.7301570	total: 1.97s	remaining: 8.23s
193:	learn: 0.7302915	total: 1.98s	remaining: 8.22s
194:	learn: 0.7301625	total: 1.99s	remaining: 8.2s
195:	learn: 0.7302244	total: 2s	remaining: 8.19s
196:	learn: 0.7302189	total: 2s	remaining: 8.18s
197:	learn: 0.7301966	

340:	learn: 0.7446010	total: 3.46s	remaining: 6.68s
341:	learn: 0.7446233	total: 3.46s	remaining: 6.67s
342:	learn: 0.7445335	total: 3.48s	remaining: 6.66s
343:	learn: 0.7445785	total: 3.48s	remaining: 6.65s
344:	learn: 0.7447582	total: 3.5s	remaining: 6.64s
345:	learn: 0.7447246	total: 3.51s	remaining: 6.63s
346:	learn: 0.7449658	total: 3.52s	remaining: 6.62s
347:	learn: 0.7452071	total: 3.53s	remaining: 6.61s
348:	learn: 0.7452072	total: 3.54s	remaining: 6.6s
349:	learn: 0.7452409	total: 3.54s	remaining: 6.58s
350:	learn: 0.7457235	total: 3.56s	remaining: 6.58s
351:	learn: 0.7458132	total: 3.56s	remaining: 6.56s
352:	learn: 0.7458975	total: 3.57s	remaining: 6.55s
353:	learn: 0.7461277	total: 3.58s	remaining: 6.54s
354:	learn: 0.7463073	total: 3.59s	remaining: 6.53s
355:	learn: 0.7463128	total: 3.6s	remaining: 6.51s
356:	learn: 0.7465094	total: 3.61s	remaining: 6.51s
357:	learn: 0.7465264	total: 3.62s	remaining: 6.5s
358:	learn: 0.7465825	total: 3.63s	remaining: 6.48s
359:	learn: 0.74

504:	learn: 0.7561914	total: 5.09s	remaining: 4.99s
505:	learn: 0.7562530	total: 5.1s	remaining: 4.98s
506:	learn: 0.7563821	total: 5.12s	remaining: 4.97s
507:	learn: 0.7563485	total: 5.13s	remaining: 4.96s
508:	learn: 0.7565055	total: 5.13s	remaining: 4.95s
509:	learn: 0.7565448	total: 5.14s	remaining: 4.94s
510:	learn: 0.7565505	total: 5.15s	remaining: 4.93s
511:	learn: 0.7566566	total: 5.16s	remaining: 4.92s
512:	learn: 0.7566850	total: 5.17s	remaining: 4.91s
513:	learn: 0.7567522	total: 5.18s	remaining: 4.9s
514:	learn: 0.7568928	total: 5.19s	remaining: 4.89s
515:	learn: 0.7567413	total: 5.2s	remaining: 4.88s
516:	learn: 0.7568422	total: 5.21s	remaining: 4.87s
517:	learn: 0.7568477	total: 5.22s	remaining: 4.86s
518:	learn: 0.7571003	total: 5.23s	remaining: 4.85s
519:	learn: 0.7572348	total: 5.24s	remaining: 4.84s
520:	learn: 0.7572181	total: 5.25s	remaining: 4.83s
521:	learn: 0.7572966	total: 5.26s	remaining: 4.82s
522:	learn: 0.7576218	total: 5.27s	remaining: 4.81s
523:	learn: 0.7

665:	learn: 0.7681326	total: 6.74s	remaining: 3.38s
666:	learn: 0.7682228	total: 6.75s	remaining: 3.37s
667:	learn: 0.7683463	total: 6.76s	remaining: 3.36s
668:	learn: 0.7685369	total: 6.78s	remaining: 3.35s
669:	learn: 0.7686154	total: 6.79s	remaining: 3.34s
670:	learn: 0.7685649	total: 6.8s	remaining: 3.33s
671:	learn: 0.7685536	total: 6.81s	remaining: 3.33s
672:	learn: 0.7685536	total: 6.83s	remaining: 3.32s
673:	learn: 0.7686039	total: 6.84s	remaining: 3.31s
674:	learn: 0.7687834	total: 6.85s	remaining: 3.3s
675:	learn: 0.7688394	total: 6.86s	remaining: 3.29s
676:	learn: 0.7688621	total: 6.88s	remaining: 3.28s
677:	learn: 0.7689124	total: 6.89s	remaining: 3.27s
678:	learn: 0.7688729	total: 6.9s	remaining: 3.26s
679:	learn: 0.7689234	total: 6.91s	remaining: 3.25s
680:	learn: 0.7690584	total: 6.92s	remaining: 3.24s
681:	learn: 0.7690194	total: 6.94s	remaining: 3.23s
682:	learn: 0.7691091	total: 6.95s	remaining: 3.23s
683:	learn: 0.7693109	total: 6.96s	remaining: 3.22s
684:	learn: 0.7

830:	learn: 0.7796503	total: 8.58s	remaining: 1.75s
831:	learn: 0.7799522	total: 8.6s	remaining: 1.74s
832:	learn: 0.7801934	total: 8.61s	remaining: 1.73s
833:	learn: 0.7801989	total: 8.62s	remaining: 1.72s
834:	learn: 0.7802607	total: 8.64s	remaining: 1.71s
835:	learn: 0.7803115	total: 8.65s	remaining: 1.7s
836:	learn: 0.7803006	total: 8.66s	remaining: 1.69s
837:	learn: 0.7804183	total: 8.67s	remaining: 1.68s
838:	learn: 0.7806148	total: 8.68s	remaining: 1.67s
839:	learn: 0.7804412	total: 8.69s	remaining: 1.65s
840:	learn: 0.7805872	total: 8.7s	remaining: 1.64s
841:	learn: 0.7806882	total: 8.71s	remaining: 1.63s
842:	learn: 0.7807387	total: 8.72s	remaining: 1.62s
843:	learn: 0.7808451	total: 8.73s	remaining: 1.61s
844:	learn: 0.7808114	total: 8.73s	remaining: 1.6s
845:	learn: 0.7807390	total: 8.74s	remaining: 1.59s
846:	learn: 0.7808008	total: 8.76s	remaining: 1.58s
847:	learn: 0.7810308	total: 8.76s	remaining: 1.57s
848:	learn: 0.7810140	total: 8.77s	remaining: 1.56s
849:	learn: 0.78

VotingClassifier(estimators=[('histgb', HistGradientBoostingClassifier()),
                             ('catb',
                              <catboost.core.CatBoostClassifier object at 0x7f886103f280>)],
                 voting='soft')

In [138]:
from sklearn.metrics import f1_score
print(f1_score(y_val,vc.predict(X_val)))

0.5771998266146511


## Test set predictions using Voting Classifier

### Preprocessing test set

In [139]:
test_data_new = test_data.copy()
for column in test_data:
    if column!='mvar47':
        test_data_new[column] = pd.to_numeric(test_data[column], errors ='coerce').fillna(-1).astype('float')
test_data_new.replace(-1, np.nan, inplace = True)

test_data_new['mvar47'] = test_data_new['mvar47'].map({'L':0, 'C':1})
test_data_new['mvar50'] = test_data_new['mvar50'].map({4:0, 5:1})
test_data_new['mvar47'] = test_data_new['mvar47'].astype(str)
test_data_new['mvar50'] = test_data_new['mvar50'].astype(str)
test_data_new.drop(drop_cols, axis=1, inplace = True)

X_test = test_data_new.copy()
X_test.drop(['application_key'], axis = 1, inplace = True)

### Predictions

In [146]:
test_preds_vc = vc.predict(X_test)
keys = test_data_new['application_key']
df_out = pd.DataFrame(list(zip(keys, test_preds_vc)), columns =['keys', 'predictions'], dtype = int)
df_out

,keys,predictions
0,578069,0
1,578070,0
2,578071,0
3,578072,0
4,578073,0
...,...,...
46995,310027,0
46996,310028,0
46997,310029,1
46998,310030,0


In [147]:
df_out.to_csv('csvs/mostly_just_bt_vc.csv', header=False, index=False)